# Sea Surface Temperature

In [ ]:
# import pickle
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from maps import map_robinson
from paths import path_results, path_samoc, file_ex_ocn_ctrl
from plotting import shifted_color_map
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA
from xr_regression import xr_linear_trends_2D, xr_linear_trend, ocn_field_regression

# global SST map and trends

In [ ]:
MASK = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).REGION_MASK

In [ ]:
%%time
# stacking files into one xr Dataset object
for run in ['ctrl', 'rcp']:
    for i, (y,m,s) in enumerate(IterateOutputCESM('ocn', run, 'yrly', name='TEMP_PD')):
        print(y)
        da = xr.open_dataset(s, decode_times=False).TEMP[0,:,:]
        if i==0:
            da_new = da.copy()
            print
        else:
            da_new = xr.concat([da_new, da], dim='time')
    da_new.to_netcdf(f'{path_samoc}/SST/SST_yrly_{run}.nc')

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc')
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' )

In [ ]:
%%time
SST_trend_ctrl = ocn_field_regression(SST_yrly_ctrl)
SST_trend_rcp  = ocn_field_regression(SST_yrly_rcp )

In [ ]:
fn = f'{path_results}/SST/SST_ctrl_mean'
xa = SST_yrly_ctrl.mean(dim='time').where(MASK>0)
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmocean.cm.thermal, minv=-2, maxv=30, label='SST [$^\circ$C]', filename=fn)

In [ ]:
cmap = shifted_color_map(cmocean.cm.balance, start=.25, midpoint=0.5, stop=1., name='shrunk')
label = 'SST trend [K/century]'

xa = SST_trend_ctrl*100
fn = f'{path_results}/SST/SST_trend_ctrl'
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4, label=label, filename=fn)

xa = SST_trend_rcp*100
fn = f'{path_results}/SST/SST_trend_rcp'
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4, label=label, filename=fn)

### hiatus trend

In [ ]:
SST_yrly_rcp[20:29,:,:]

In [ ]:
SST_trend_rcp_hiatus  = ocn_field_regression(SST_yrly_rcp[20:29,:,:])

In [ ]:
fn = f'{path_results}/SST/SST_trend_rcp_hiatus'
cmap = shifted_color_map(cmocean.cm.balance, start=.25, midpoint=0.5, stop=1., name='shrunk')
label = 'SST trend [K/century]'
xa = SST_trend_rcp_hiatus*100
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-4, maxv=8, label=label, filename=fn)

# Southern Ocean
## Southern Ocean Mode index

In [ ]:
AREA_rect = xr_AREA('ocn_rect')
AREA_rect = AREA_rect.rename({'t_lon':'lon', 't_lat':'lat'})

In [ ]:
from regions import SOM_area_rect
SOM_index_area = AREA_rect.sel(SOM_area_rect).sum()

In [ ]:
SOM_index_ctrl = (AREA_rect*SST_yrly_ctrl).sel(SOM_area_rect).sum(dim=('lat', 'lon'))/SOM_index_area
SOM_index_rcp  = (AREA_rect*SST_yrly_rcp ).sel(SOM_area_rect).sum(dim=('lat', 'lon'))/SOM_index_area

In [ ]:
SOM_index_ctrl_trend = xr_linear_trend(SOM_index_ctrl)
SOM_index_rcp_trend  = xr_linear_trend(SOM_index_rcp )

times_ctrl = np.arange(len(SOM_index_ctrl))
times_rcp  = np.arange(len(SOM_index_rcp ))

SOM_index_ctrl_detr = SOM_index_ctrl - times_ctrl*SOM_index_ctrl_trend.item()
SOM_index_rcp_detr  = SOM_index_rcp  - times_rcp *SOM_index_rcp_trend.item()

SOM_index_ctrl = SOM_index_ctrl_detr - SOM_index_ctrl_detr.mean()
SOM_index_rcp  = SOM_index_rcp_detr  - SOM_index_rcp_detr.mean() 

In [ ]:
f = plt.figure(figsize=(8,5))
ax = f.add_axes([0.13,0.13,.85,.85])
plt.tick_params(labelsize=14)

plt.axhline(0,c='k', lw=.5)
plt.plot(SOM_index_ctrl, c='C0', lw=.5)
plt.plot(SOM_index_rcp , c='C1', lw=.5)
plt.plot(SOM_index_ctrl.rolling({'time':5}, center=True).mean(), c='C0', lw=2, label='SOM index CTRL')
plt.plot(SOM_index_rcp.rolling( {'time':5}, center=True).mean(), c='C1', lw=2, label='SOM index RCP')

plt.text(.02,.95, r'[35,50$^\circ$S]$\times$[-50,0$^\circ$E]', ha='left',transform=ax.transAxes, fontsize=16)
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('SOM index [K]', fontsize=16)
plt.legend(fontsize=16, ncol=2)
plt.savefig(f'{path_results}/SST/SOM_index')

# Pacific Ocean

## Interdecadal Pacific Oscillation

## Pacific decadal oscillation
"The leading EOF of monthlySST anomalies over the North Pacific (after removing the global mean SST anomaly) and its associated PC time series are termed the Pacific Decadal Oscillation (PDO)"

# Atlantic Ocean
## Atlantic Multidecadal Oscillation